In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [31]:
#LIST_PATH from json
json_path = '/home/deeplearning/AHL/AHL_new_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [32]:
dataset[0]

['/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3',
 'CT',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441',
 'PET0',
 '13011101051024']

#list path from csv 
csv_path= '/home/deeplearning/AHL/conditions_suv.csv'
dataset = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    dataset = []
    for row in reader :
        dataset.append(row)
        
del dataset[0] #enlever première ligne

In [33]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

number of study :  231


In [34]:
#add csv_file to dataset 

csv_path= '/home/deeplearning/AHL/AHL_CSV_DATA/pet0.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [35]:
for data in dataset : 
    uid = data[-3]

    for row in csv_file: 
        if uid in row : 
            data.append(os.path.join( '/home/deeplearning/AHL/AHL_CSV_DATA', row[4]))

In [36]:
dataset[0]

['/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3',
 'CT',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441',
 'PET0',
 '13011101051024',
 '/home/deeplearning/AHL/AHL_CSV_DATA/tab_do_nov 4_2013.csv']

In [40]:
#pet2 & pet 4 

pet_2 = []
pet_4 = []
for data in dataset : 
    if 'pet2' in data[5].lower():
        pet_2.append(data)

for data in dataset : 
    if 'pet4' in data[5].lower():
        pet_4.append(data)


print(len(pet_2))
print(len(pet_4))

51
41


In [10]:
#remove PET 2 and PET 4
remove_study = []
for data in dataset : 
    if len(data) != 8 : 
        remove_study.append(data)

for r in remove_study : 
    dataset.remove(r)

In [11]:
print('Nombre de study : ', len(dataset))

Nombre de study :  128


In [40]:
#remove remove_bi_file if it exists in every serie 
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

In [ ]:
#Check if there is Unconstant Spacing error in every serie of the dataset 
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


In [ ]:
#Careful : maybe need to rewrite path if the series are in an other folder

In [17]:
for serie in dataset : 
    if len(serie) != 8 : 
        print(serie)

In [19]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'
mip_directory = '/home/deeplearning/AHL/AHL_MIP'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv file  
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 7) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 1.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")

            else : 
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask with threshold in csv 
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                #create mip for false mask and check 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin = 0, vmax = 7) 
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys', borne_max = 1.0)
                subliste.append(angle_filename_mask)
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

8, 8, 'ELLIPSE', 0.5599999999999996, 9, 'ELLIPSE', 0.71, 10, 'ELLIPSE', 0.5300000000000002, 11, 'ELLIPSE', 0.5300000000000002, 12, 'ELLIPSE', 0.5700000000000003, 13, 'ELLIPSE', 0.54, 14, 'ELLIPSE', 1.3499999999999996, 15, 'ELLIPSE', 0.5899999999999999, 16, 'ELLIPSE', 0.8499999999999996, 17, 'ELLIPSE', 0.6699999999999999, 18, 'ELLIPSE', 0.6800000000000006, 19, 'ELLIPSE', 1.2899999999999991, 20, 'ELLIPSE', 0.4400000000000004, 21, 'ELLIPSE', 0.49999999999999956, 22, 'ELLIPSE', 0.5899999999999999, 23, 'ELLIPSE', 0.8399999999999999, 24, 'ELLIPSE', 0.7999999999999998, 25, 'ELLIPSE', 0.5600000000000005, 26, 'ELLIPSE', 0.7299999999999995, 27, 'ELLIPSE', 1.1799999999999997, 28, 'ELLIPSE', 1.709999999999999, 29, 'ELLIPSE', 1.0999999999999996, 30, 'ELLIPSE', 0.79, 31, 'ELLIPSE', 1.3699999999999992, 32, 'ELLIPSE', 0.6900000000000004, 33, 'ELLIPSE', 0.6899999999999995, 34, 'ELLIPSE', 0.8099999999999996, 35, 'ELLIPSE', 0.7000000000000002, 36, 'ELLIPSE', 0.5200000000000005, 37, 'ELLIPSE', 0.879999999

In [21]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

number of error :  7
number of false mask :  52


In [22]:
dataset[3]

['/home/deeplearning/AHL/AHL_Validated_DICOM/13011101391002/PET2/1.2.250.1.59.453.859.92444169.2200.1329394379.6.2.9',
 'PT',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101391002/PET2/1.2.250.1.59.453.859.92444169.2200.1329394379.6.2.10',
 'CT',
 '13011101391002',
 'PET2',
 '13011101391002',
 '/home/deeplearning/AHL/AHL_CSV_DATA/gra_re_jan 4_2012.csv']

In [46]:
#Save different json 
directory_nifti = ''
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)

write_json_file(directory_nifti, 'error', error_dataset)

mip directory = directory_nifti+'/'+'MIP'
write_json_file(mip_directory, 'path_mip', path_mip)

In [47]:
#create pdf of MIP for false mask 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [26]:
id_ = ['13011101391002', '13011101391005']
remove_liste = []
for i in id_ : 
    for data in dataset : 
        if i in data : 
            remove_liste.append(data)
#print(len(remove_liste))

for r in remove_liste : 
    dataset.remove(r)

2


In [27]:
#Generate PET, CT, MASK NIFTI without checking

nifti_directory = nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

0
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. GDCM)
['/home/deeplearning/AHL/AHL_Validated_DICOM/13011101151005/pet0/1.3.12.2.1107.5.1.4.44873.30000013040806565018700003254', 'CT', '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101151005/pet0/1.3.12.2.1107.5.1.4.44873.30000013040805421781200000023', 'PT', '1.3.12.2.1107.5.1.4.44873.30000013040805421771800000040', 'PET0', '13011101151005', '/home/deeplearning/AHL/AHL_CSV_DATA/sim_ch_apr 8_2013.csv']
2
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
index 160 is out of bounds for axis 2 with size 87
['/home/deeplearning/AHL/AHL_Validated_DICOM/53011101371002/PET0/1.2.840.113619.2.55.3.413972756.569.1340604297.279.3', 'CT', '/home/deeplearning/AHL/AHL_Validated_DICOM/53011101371002/PET2/1.2.840.113619.2.131.413972756.1340635377.171331', 'PT', '1.2.250.1.74.20120625133000.100003136

In [43]:
#Generate PET & CT NIFTI only

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI/pet4'

error = []
for serie in pet_4 : 
    print(pet_4.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

0
EXPORT NIFTI PT
EXPORT NIFTI CT
1
EXPORT NIFTI PT
EXPORT NIFTI CT
2
EXPORT NIFTI PT
EXPORT NIFTI CT
3
EXPORT NIFTI PT
EXPORT NIFTI CT
4
EXPORT NIFTI PT
EXPORT NIFTI CT
5
EXPORT NIFTI PT
EXPORT NIFTI CT
6
EXPORT NIFTI PT
EXPORT NIFTI CT
7
EXPORT NIFTI PT
EXPORT NIFTI CT
8
EXPORT NIFTI PT
EXPORT NIFTI CT
9
EXPORT NIFTI PT
EXPORT NIFTI CT
10
EXPORT NIFTI PT
EXPORT NIFTI CT
11
EXPORT NIFTI PT
EXPORT NIFTI CT
12
EXPORT NIFTI PT
EXPORT NIFTI CT
13
EXPORT NIFTI PT
EXPORT NIFTI CT
14
EXPORT NIFTI PT
EXPORT NIFTI CT
15
EXPORT NIFTI PT
EXPORT NIFTI CT
16
EXPORT NIFTI PT
EXPORT NIFTI CT
17
EXPORT NIFTI PT
EXPORT NIFTI CT
18
EXPORT NIFTI PT
EXPORT NIFTI CT
19
EXPORT NIFTI PT
EXPORT NIFTI CT
20
EXPORT NIFTI PT
EXPORT NIFTI CT
21
EXPORT NIFTI PT
EXPORT NIFTI CT
22
EXPORT NIFTI PT
EXPORT NIFTI CT
23
EXPORT NIFTI PT
EXPORT NIFTI CT
24
EXPORT NIFTI PT
EXPORT NIFTI CT
25
EXPORT NIFTI PT
EXPORT NIFTI CT
26
EXPORT NIFTI PT
EXPORT NIFTI CT
27
EXPORT NIFTI PT
EXPORT NIFTI CT
28
EXPORT NIFTI PT
EXPORT NIFT

In [5]:
#Generate PET only

nifti_directory = '/home/deeplearning/AHL/suv_test'
dataset = dataset[0:10]

error = []
for serie in dataset : 
    print(dataset.index(serie))
    print(serie[0])
    try : 
                serie_pt_objet = SeriesPT(serie[1]) 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)

0
GE MEDICAL SYSTEMS
EXPORT NIFTI PT
1
PHILIPS_CNTS
EXPORT NIFTI PT
2
PHILIPS_BQML
EXPORT NIFTI PT
3
CPS
EXPORT NIFTI PT
4
BQML
EXPORT NIFTI PT
5
PROPCNTS
Error generating result array Unknown PET Units
Error generating result array Unknown PET Units
['PROPCNTS', '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101251027/pet0/1.2.840.113619.2.131.444444.1390920991.901647']
axes don't match array
6
PHILIPS_CNTS_BQML_SUV_OK
EXPORT NIFTI PT
7
PHILIPS_CNTS_SUV_OK
EXPORT NIFTI PT
8
RADIOPHARM_DATETIME_UNDEFINED
EXPORT NIFTI PT
9
BQML_AC_DT_<_S_DT
EXPORT NIFTI PT
